In [55]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import datetime
import yfinance as yfin

from var import VaR

pd.set_option('display.float_format', lambda x: f'{x:,.4f}')

In [56]:
start = datetime.datetime(2023, 9, 11)
end = datetime.datetime(2023, 9, 12)


#start_date = "2022-01-01"
#end_date = "2023-08-31"

In [57]:
folio_df = pd.read_csv('./data/DATA618_Portfolio.csv', 
                 dtype={
                    'ID': 'int',
                    'Price': 'float',
                    'Shares': 'float',
                    'Value': 'float', 
                 })


folio_df = folio_df[folio_df['Shares'] > 0]
#folio_df = folio_df.groupby(['Ticker','BondName','Class'])[['Shares','Value']].sum()
folio_df.reset_index(inplace=True)
#folio_df['Morningstar_Category'].fillna('Other', inplace=True)
folio_df

,index,Unnamed: 0,Ticker,BondName,Morningstar_Category,Class,Sub_Class,Price,Shares,Value
0,0,0,IVV,iShares Core S&P 500 ETF,Large Blend,Economically Sensitive,U.S. Equities,448.8456,"8,912.0000","4,000.0000"
1,1,1,VONG,Vanguard Russell 1000 Growth ETF,Large Growth,Economically Sensitive,U.S. Equities,72.2877,"41,501.0000","3,000.0000"
2,2,2,SCHD,Schwab US Dividend Equity ETF™,Large Value,Economically Sensitive,U.S. Equities,73.1219,"41,027.0000","3,000.0000"
3,3,3,IMCB,iShares Morningstar Mid-Cap ETF,Mid-Cap Blend,Economically Sensitive,U.S. Equities,62.0215,"64,494.0000","4,000.0000"
4,4,4,VO,Vanguard Mid-Cap ETF,Mid-Cap Blend,Economically Sensitive,U.S. Equities,216.5903,"9,234.0000","2,000.0000"
...,...,...,...,...,...,...,...,...,...,...
30,30,35,BPLSX,Boston Partners Long/Short Equity Instl,Other,Economically Sensitive,Long/Short Equities,15.0100,"49,967.0000",750.0000
31,31,36,GARIX,Gotham Absolute Return Institutional,Other,Economically Sensitive,Long/Short Equities,19.7800,"25,278.0000",500.0000
32,32,37,GCHDX,Gotham Hedged Core Institutional,Other,Economically Sensitive,Long/Short Equities,10.8200,"46,211.0000",500.0000
33,33,38,BLUEX,AMG Veritas Global Real Return I,Other,Economically Sensitive,Long/Short Equities,35.2100,"14,201.0000",500.0000


In [58]:
list(folio_df['Ticker'])

['IVV',
 'VONG',
 'SCHD',
 'IMCB',
 'VO',
 'FSMD',
 'VBR',
 'VIGI',
 'JPEM',
 'ISCF',
 'IGRO',
 'VNQ',
 'DFAR',
 'FREL',
 'JPRE',
 'MMIN',
 'MEAR',
 'MMIT',
 'SNDPX',
 'ALTVX',
 'IGMWX',
 'VFIUX',
 'VICSX',
 'VGCAX',
 'PSAIX',
 'PFRAX',
 'SWRSX',
 'ICBFX',
 'VTEB',
 'PEX',
 'BPLSX',
 'GARIX',
 'GCHDX',
 'BLUEX',
 'C_A_S_H']

In [59]:
yfin.pdr_override()

df = pdr.get_data_yahoo(list(folio_df['Ticker']), start=start, end=end)
df

[*********************100%%**********************]  35 of 35 completed


1 Failed download:
['C_A_S_H']: Exception('%ticker%: No timezone found, symbol may be delisted')


Adj Close                                                  \
                        ALTVX   BLUEX   BPLSX C_A_S_H    DFAR    FREL    FSMD   
Date                                                                            
2023-09-11 00:00:00    9.4000 35.2100 15.0100     NaN 20.7436 24.2468 33.1387   

                     ... Volume                                         \
                     ...  VGCAX VICSX    VIGI      VNQ      VO    VONG   
Date                 ...                                                 
2023-09-11 00:00:00  ...      0     0  200100  2813600  424900  731900   

                              
                        VTEB  
Date                          
2023-09-11 00:00:00  5068900  

[1 rows x 210 columns]

In [60]:
#df.columns

df['C_A_S_H'] = 1
df[['C_A_S_H']]

In [61]:
df = df[['Adj Close']]
df.columns = df.columns.get_level_values(1)
df['C_A_S_H'] = 1
df

,ALTVX,BLUEX,BPLSX,C_A_S_H,DFAR,FREL,FSMD,...,VGCAX,VICSX,VIGI,VNQ,VO,VONG,VTEB
Date,,,,,,,,,,,,,,,
2023-09-11 00:00:00,9.4000,35.2100,15.0100,1,20.7436,24.2468,33.1387,...,18.3700,20.8500,74.1083,80.8038,216.5903,72.2877,49.0532


In [62]:
p_df = df.T
p_df.columns = ['Price2']
p_df.reset_index(inplace=True)
p_df.columns = ['Ticker','Price2']
p_df.to_csv('./data/p.csv', index=False)

In [63]:
folio_df = folio_df.merge(p_df, how='left', left_on='Ticker', right_on='Ticker' )
folio_df

,index,Unnamed: 0,Ticker,BondName,Morningstar_Category,Class,Sub_Class,Price,Shares,Value,Price2
0,0,0,IVV,iShares Core S&P 500 ETF,Large Blend,Economically Sensitive,U.S. Equities,448.8456,"8,912.0000","4,000.0000",448.8456
1,1,1,VONG,Vanguard Russell 1000 Growth ETF,Large Growth,Economically Sensitive,U.S. Equities,72.2877,"41,501.0000","3,000.0000",72.2877
2,2,2,SCHD,Schwab US Dividend Equity ETF™,Large Value,Economically Sensitive,U.S. Equities,73.1219,"41,027.0000","3,000.0000",73.1219
3,3,3,IMCB,iShares Morningstar Mid-Cap ETF,Mid-Cap Blend,Economically Sensitive,U.S. Equities,62.0215,"64,494.0000","4,000.0000",62.0215
4,4,4,VO,Vanguard Mid-Cap ETF,Mid-Cap Blend,Economically Sensitive,U.S. Equities,216.5903,"9,234.0000","2,000.0000",216.5903
...,...,...,...,...,...,...,...,...,...,...,...
30,30,35,BPLSX,Boston Partners Long/Short Equity Instl,Other,Economically Sensitive,Long/Short Equities,15.0100,"49,967.0000",750.0000,15.0100
31,31,36,GARIX,Gotham Absolute Return Institutional,Other,Economically Sensitive,Long/Short Equities,19.7800,"25,278.0000",500.0000,19.7800
32,32,37,GCHDX,Gotham Hedged Core Institutional,Other,Economically Sensitive,Long/Short Equities,10.8200,"46,211.0000",500.0000,10.8200
33,33,38,BLUEX,AMG Veritas Global Real Return I,Other,Economically Sensitive,Long/Short Equities,35.2100,"14,201.0000",500.0000,35.2100


In [64]:
folio_df['Price'] = folio_df['Price2']

In [65]:
folio_df['check'] = folio_df['Value'] * 1000 / folio_df['Price']

In [66]:
folio_df['check'] =  round(folio_df['check'],0)
folio_df['check2'] = folio_df['check'] * folio_df['Price'] / 1000

In [67]:
folio_df['check2'].sum()

83719.05870270061

In [68]:
folio_df['Shares'] = folio_df['check'] 
folio_df.drop(labels=['Price2','check','check2'], inplace=True, axis=1)
folio_df

,index,Unnamed: 0,Ticker,BondName,Morningstar_Category,Class,Sub_Class,Price,Shares,Value
0,0,0,IVV,iShares Core S&P 500 ETF,Large Blend,Economically Sensitive,U.S. Equities,448.8456,"8,912.0000","4,000.0000"
1,1,1,VONG,Vanguard Russell 1000 Growth ETF,Large Growth,Economically Sensitive,U.S. Equities,72.2877,"41,501.0000","3,000.0000"
2,2,2,SCHD,Schwab US Dividend Equity ETF™,Large Value,Economically Sensitive,U.S. Equities,73.1219,"41,027.0000","3,000.0000"
3,3,3,IMCB,iShares Morningstar Mid-Cap ETF,Mid-Cap Blend,Economically Sensitive,U.S. Equities,62.0215,"64,494.0000","4,000.0000"
4,4,4,VO,Vanguard Mid-Cap ETF,Mid-Cap Blend,Economically Sensitive,U.S. Equities,216.5903,"9,234.0000","2,000.0000"
...,...,...,...,...,...,...,...,...,...,...
30,30,35,BPLSX,Boston Partners Long/Short Equity Instl,Other,Economically Sensitive,Long/Short Equities,15.0100,"49,967.0000",750.0000
31,31,36,GARIX,Gotham Absolute Return Institutional,Other,Economically Sensitive,Long/Short Equities,19.7800,"25,278.0000",500.0000
32,32,37,GCHDX,Gotham Hedged Core Institutional,Other,Economically Sensitive,Long/Short Equities,10.8200,"46,211.0000",500.0000
33,33,38,BLUEX,AMG Veritas Global Real Return I,Other,Economically Sensitive,Long/Short Equities,35.2100,"14,201.0000",500.0000


In [69]:
folio_df.drop(labels=['index'], inplace=True, axis=1)
folio_df

,Unnamed: 0,Ticker,BondName,Morningstar_Category,Class,Sub_Class,Price,Shares,Value
0,0,IVV,iShares Core S&P 500 ETF,Large Blend,Economically Sensitive,U.S. Equities,448.8456,"8,912.0000","4,000.0000"
1,1,VONG,Vanguard Russell 1000 Growth ETF,Large Growth,Economically Sensitive,U.S. Equities,72.2877,"41,501.0000","3,000.0000"
2,2,SCHD,Schwab US Dividend Equity ETF™,Large Value,Economically Sensitive,U.S. Equities,73.1219,"41,027.0000","3,000.0000"
3,3,IMCB,iShares Morningstar Mid-Cap ETF,Mid-Cap Blend,Economically Sensitive,U.S. Equities,62.0215,"64,494.0000","4,000.0000"
4,4,VO,Vanguard Mid-Cap ETF,Mid-Cap Blend,Economically Sensitive,U.S. Equities,216.5903,"9,234.0000","2,000.0000"
...,...,...,...,...,...,...,...,...,...
30,35,BPLSX,Boston Partners Long/Short Equity Instl,Other,Economically Sensitive,Long/Short Equities,15.0100,"49,967.0000",750.0000
31,36,GARIX,Gotham Absolute Return Institutional,Other,Economically Sensitive,Long/Short Equities,19.7800,"25,278.0000",500.0000
32,37,GCHDX,Gotham Hedged Core Institutional,Other,Economically Sensitive,Long/Short Equities,10.8200,"46,211.0000",500.0000
33,38,BLUEX,AMG Veritas Global Real Return I,Other,Economically Sensitive,Long/Short Equities,35.2100,"14,201.0000",500.0000


In [70]:
folio_df.to_csv('./data/tmp.csv', index=False)

In [71]:
t_df = folio_df.copy()
t_df['Value2'] = t_df['Price'] * t_df['Shares']
t_df['Value2'].sum()

83719058.70270061

folio_df['check'] = folio_df['Price'] - folio_df['Price2']
folio_df.sort_values(by='check', ascending=False)